# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: Carlos Mario Padilla Gomez  <br>
Url: https://github.com/padillacm/03MAIR---Algoritmos-de-Optimizacion <br>
Problema:
> 1. Sesiones de doblaje <br>
>2. Organizar los horarios de partidos de La Liga<br>
>3. Combinar cifras y operaciones

Descripción del problema:(copiar enunciado)

....

(*) La respuesta es obligatoria





                                        

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible.

(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>



¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.




Respuesta

(30C6)x(24C6)x(18C6)x(12C6)x(6C6)

Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Respuesta

En el caso del algoritmo genético inicialmente tenía una estructura de 4 dimensiones con la que modelar la cantidad de población, luego correspondía a los genes, luego a los alelos y cada alelo el índice de las tomas que le corresponden en el día. Por otra parte, tenía una matriz que contenía el índice de cada gen y el pago del rodaje y la cantidad de intersecciones. Al final vi que era ineficiente tener estas dos por separado así que junte todo en una estructura que tiene [num poblacion,pago rodaje,intersecciones, gen], ya con este arreglo podía hacer todo dentro del algoritmo genético. Mientras que con el algoritmo de fuerza bruta solo es necesario una lista por cada día que contiene el índice de las tomas.

Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

Respuesta

En el caso del problema se desea principalmente minimizar la cantidad a pagar por todo el rodaje y para ello tambien maximixar la cantidad de tomas que un actor tiene que rodar durante el día.

In [26]:
## una funcion de utilidad
import numpy as np
import random
import copy

def calculate_repetitions_and_payment(tomasDia):
    count_actores = np.count_nonzero(tomasDia,axis=0)
    mask = (count_actores - 1) > 0
    return np.sum(count_actores[mask]-1),np.count_nonzero(count_actores)

Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

In [ ]:
from itertools import combinations
m = 30
pagoMejor,councidenciaMejor = 99999999,999999999
def remove_tuple(list_, tuple_):
    for t in tuple_:
        list_.remove(t)
    return list_

list_1 = [i for i in range(m)]

combs_1 = combinations(list_1, 6)
for comb_1 in combs_1:
    councidencia1,pago1 = calculate_repetitions_and_payment(cronograma_tomas[comb_1,:])
    list_2 = list_1.copy()
    list_2 = remove_tuple(list_2, comb_1)
    combs_2 = combinations(list_2, 6)
    for comb_2 in combs_2:
        councidencia2,pago2 = calculate_repetitions_and_payment(cronograma_tomas[comb_2,:])
        list_3 = list_2.copy()
        list_3 = remove_tuple(list_3, comb_2)
        combs_3 = combinations(list_3, 6)
        for comb_3 in combs_3:
            councidencia3,pago3 = calculate_repetitions_and_payment(cronograma_tomas[comb_3,:])
            list_4 = list_3.copy()
            list_4 = remove_tuple(list_4, comb_3)
            combs_4 = combinations(list_4, 6)
            for comb_4 in combs_4:
                councidencia4,pago4 = calculate_repetitions_and_payment(cronograma_tomas[comb_4,:])
                list_5 = list_4.copy()
                list_5 = remove_tuple(list_5, comb_4)
                combs_5 = combinations(list_5, 6)
                for comb_5 in combs_5:
                    councidencia5,pago5 = calculate_repetitions_and_payment(cronograma_tomas[comb_5,:])
                    #print(comb_1, comb_2, comb_3, comb_4, comb_5)
                    councidenciaTotal = councidencia1+councidencia2+councidencia3+councidencia4+councidencia5
                    pagoTotal = pago1+pago2+pago3+pago4+pago5
                    if (pagoTotal < pagoMejor) or (pagoTotal == pagoMejor or councidenciaTotal < councidenciaMejor):
                            pagoMejor = pagoTotal
                            councidenciaMejor = councidenciaTotal
                            mejorCronograma = (comb_1, comb_2, comb_3, comb_4, comb_5)
print(pagoMejor,councidenciaMejor,mejorCronograma)

Calcula la complejidad del algoritmo por fuerza bruta

Respuesta

sea n la cantidad de tomas y k la maxima cantidad de tomas por días.

$$\mathcal{O} \left ( \prod_{i = 0}^{\left \lceil \frac{n}{k} \right \rceil - 1}{(n - k \cdot i) \choose \min (k, n - k \cdot i)} \right )$$

(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

En este caso se desarrolló un algoritmo genético como solución no determinista a mejorar un problema que tiene un espacio de búsqueda muy grande. La ventaja de este tipo de algoritmos es que podemos tener un buen resultado, no necesariamente el mejor, pero en un tiempo mucho mas reducido que el de fuerza bruta donde son tantas combinaciones posibles que en tiempo no es viable. Además, la ventaja del algoritmo genético es que nos permite flexibilizar el problema para cuando se sumen mas restricciones, haciéndolas más fácil de implementar y aun así obteniendo un resultado aceptable.   

In [17]:
def inicializacion(numDias,numTomasPorDia,num_total_tomas,n_poblacion,tomas_disponibles):
    genes = [[] for i in range(n_poblacion)]   
    for i in range(n_poblacion):
        temp = np.asarray(tomas_disponibles.copy())
        cronograma = []
        for dia in range(numDias):
            sub = random.sample(list(temp), numTomasPorDia)
            mask = np.ones(len(temp), dtype=bool)
            pos = np.where((temp == sub[0]) | (temp == sub[1]) | 
             (temp == sub[2]) | (temp == sub[3]) | 
             (temp == sub[4]) | (temp == sub[5]))
            mask[pos] = False
            temp = temp[mask]
            cronograma.append(sub)
        genes[i].append(0)
        genes[i].append(0)
        genes[i].append(cronograma)
    return np.asarray(genes)

In [18]:
def fitness(n_poblacion,numDias,poblacion):
    for i in range(n_poblacion):
        pago_total = 0
        total_coincidencia = 0
        for dia in range(numDias):
            toma_dia = poblacion[i,2][dia]
            resultado_dia = calculate_repetitions_and_payment(cronograma_tomas[toma_dia])
            pago_total += resultado_dia[1]
            total_coincidencia += resultado_dia[0]
        poblacion[i,0] = pago_total
        poblacion[i,1] = total_coincidencia
    return poblacion

In [19]:
def selection(num_aptos,poblacion):
    poblacion = list(poblacion)
    poblacion.sort(key=lambda x: (x[0],-1*x[1]))
    poblacion = np.asarray(poblacion)
    return (list(poblacion[0:num_aptos,:]),poblacion)

In [20]:
def corrector_cruce(gen,numDias,numTomasPorDia):
    gen_evaluation = [[i,calculate_repetitions_and_payment(cronograma_tomas[gen[i]])] for i in range(numDias)]
    gen_evaluation.sort(key=lambda x: (-1*x[1][1],x[1][0]))

    values,count = np.unique(gen,return_counts = True)
    faltantes = list(set(tomas_disponibles) - set(values))
    repetidos = list(values[count > 1])
    count_repetidos = list(count[count > 1])

    for i,score in gen_evaluation:
        num_sobrantes = set(gen[i]).intersection(set(repetidos))
        if(len(repetidos) == 0):
                break
        for j in range(numTomasPorDia):
            if(len(num_sobrantes) == 0):
                break
            if gen[i][j] in num_sobrantes:
                tempNum = gen[i][j]
                newNum = random.choice(faltantes)
                gen[i][j] = newNum

                faltantes.remove(newNum)
                num_sobrantes.remove(tempNum)

                index_num_repetido = repetidos.index(tempNum)
                count_repetidos[index_num_repetido] -= 1

                if count_repetidos[index_num_repetido] == 1:
                    repetidos.pop(index_num_repetido)
                    count_repetidos.pop(index_num_repetido)
    return gen

In [21]:
def reproduccion(numDias,numTomasPorDia,num_actos,mas_actos,poblacion):
    for i in range (num_actos,poblacion.shape[0]):
        point = 0
        point = np.random.randint(1,numDias - 1)
        padres = random.sample(mas_actos,2)

        gen = poblacion[i,2]
        gen[:point] = copy.deepcopy(padres[0][2][:point])
        gen[point:] = copy.deepcopy(padres[1][2][point:])
        gen = corrector_cruce(gen,numDias,numTomasPorDia)
    return poblacion

In [22]:
def mutacion(numTomasPorDia,num_alelos_tot,num_max_mutaciones,porcentaje_mutacion,poblacion):
    for _,_,gen in poblacion:
        if np.random.choice((True, False), p=[porcentaje_mutacion, 1-porcentaje_mutacion]):
            num_mutaciones = np.random.randint(2,num_max_mutaciones+1)
            alelos_select = random.sample(num_alelos_tot,num_mutaciones)
            temp = np.array(gen)[alelos_select].flatten()
            for num_alelo in alelos_select:
                new_alelo = random.sample(list(temp), numTomasPorDia)
                mask = np.ones(len(temp), dtype=bool)
                pos = np.where((temp == new_alelo[0]) | (temp == new_alelo[1]) | 
                 (temp == new_alelo[2]) | (temp == new_alelo[3]) | 
                 (temp == new_alelo[4]) | (temp == new_alelo[5]))
                mask[pos] = False
                temp = temp[mask]
                gen[num_alelo] = new_alelo
    return poblacion

In [45]:
numDias = 5
numTomasPorDia = 6
num_total_tomas = 30
n_poblacion = 100
num_actos = 7
porcentaje_mutacion = 0.3
num_max_mutaciones = 3
tomas_disponibles = [i for i in range(num_total_tomas)]
num_alelos_tot = [i for i in range(numDias)]
mejor_gen = [[99999,99999]]

In [38]:
cronograma_tomas_temp = []
with open('tomas.txt') as f:
    lines = f.readlines()
    for line in lines:
        cronograma_tomas_temp.append((line.replace('\n','').split('\t')))
cronograma_tomas = np.asarray(cronograma_tomas_temp,dtype=np.uint8)

In [28]:
totGeneraciones = 500
poblacion = inicializacion(numDias,numTomasPorDia,num_total_tomas,n_poblacion,tomas_disponibles)
for generacion in range(totGeneraciones):
    
    poblacion = fitness(n_poblacion,numDias,poblacion)
    mas_actos, poblacion = selection(num_actos,poblacion)
    
    if mas_actos[0][0] < mejor_gen[0][0] or (mas_actos[0][0] == mejor_gen[0][0] and mas_actos[0][1] > mejor_gen[0][1]):
        mejor_gen[0] = mas_actos[0]
        
    poblacion = reproduccion(numDias,numTomasPorDia,num_actos,mas_actos,poblacion)
    poblacion = mutacion(numTomasPorDia,num_alelos_tot,num_max_mutaciones,porcentaje_mutacion,poblacion)

In [29]:
print("pago total: ",mejor_gen[0][0])
print("total intersecciones: ",mejor_gen[0][1])
print("distribucion: ",mejor_gen[0][2])

pago total:  28
total intersecciones:  66
distribucion:  [[7, 25, 28, 11, 6, 9], [27, 0, 1, 21, 16, 19], [24, 8, 15, 5, 29, 26], [18, 17, 20, 13, 23, 22], [3, 14, 2, 10, 4, 12]]


Mejor resultado obtenido hasta el momento  
pago total:  27  
total intersecciones:  67  
distribucion:  [[5, 8, 12, 26, 29, 27], [14, 7, 20, 2, 28, 3], [11, 25, 9, 6, 10, 4], [24, 21, 0, 19, 1, 15], [18, 16, 22, 17, 23, 13]]  

(*)Calcula la complejidad del algoritmo 

Respuesta

luego de analizar todo el codigo, el proceso mas pesado de todos es el de la reproduccion y su correccion llevando que la complejidad sea:  

$$ \begin{equation}
    p = \text{número de población} \\
    a = \text{número de genes actos} \\
    d = \text{número de días del rodaje} \\
    t = \text{número de tomas por día} \\
    g = \text{número de generaciones} \\
\end{equation} $$

$$\mathcal{O} \left ({(p - a)*d*t*g} \right )$$

Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

In [50]:
cronograma_tomas_temp = []
for i in range(numDias*numTomasPorDia):
    cronograma_tomas_temp.append(np.random.choice([0, 1], size=(12,), p=[0.5, 0.5])) 
cronograma_tomas = np.asarray(cronograma_tomas_temp,dtype=np.uint8)
print(cronograma_tomas)

[[0 0 0 1 0 0 0 0 1 1 0 0]
 [1 0 1 1 1 1 1 0 0 0 1 1]
 [1 0 0 1 0 0 1 0 0 0 0 1]
 [1 0 0 1 1 1 1 1 0 0 1 0]
 [0 1 1 1 0 0 1 0 0 1 1 0]
 [1 0 1 1 1 1 0 1 0 1 1 1]
 [1 0 0 1 0 1 0 0 1 0 1 0]
 [0 0 0 0 0 1 1 1 0 1 1 1]
 [0 0 1 1 0 1 0 1 1 1 0 1]
 [1 1 0 0 0 1 0 0 1 1 0 0]
 [0 1 0 0 0 0 1 0 1 0 0 1]
 [1 0 0 0 1 0 0 1 0 1 1 1]
 [0 1 1 0 1 1 0 1 0 1 1 1]
 [1 0 0 1 1 0 1 1 0 0 0 1]
 [1 1 1 1 0 1 1 0 1 1 0 0]
 [1 1 0 1 0 1 1 0 0 0 0 1]
 [0 0 0 0 1 0 1 0 0 0 1 0]
 [1 1 0 0 0 1 1 0 1 0 0 0]
 [0 1 1 1 0 0 1 0 0 0 0 0]
 [1 0 0 0 1 0 1 0 0 0 1 0]
 [1 0 1 0 1 0 1 0 1 1 1 1]
 [1 1 0 0 0 1 0 0 0 1 0 1]
 [1 0 0 1 1 0 0 1 0 0 0 1]
 [1 0 0 1 0 1 0 0 1 0 0 0]
 [0 0 1 0 1 1 0 1 0 1 0 0]
 [0 0 1 1 1 1 0 0 1 1 0 0]
 [0 1 1 0 0 1 0 1 1 1 0 1]
 [1 1 1 1 0 0 1 0 1 0 0 0]
 [0 1 1 0 0 1 0 1 1 1 1 1]
 [1 1 0 1 0 0 0 0 0 0 1 0]]


Aplica el algoritmo al juego de datos generado

Respuesta

In [51]:
totGeneraciones = 1000
mejor_gen = [[99999,99999]]

poblacion = inicializacion(numDias,numTomasPorDia,num_total_tomas,n_poblacion,tomas_disponibles)
for generacion in range(totGeneraciones):
    
    poblacion = fitness(n_poblacion,numDias,poblacion)
    mas_actos, poblacion = selection(num_actos,poblacion)
    
    if mas_actos[0][0] < mejor_gen[0][0] or (mas_actos[0][0] == mejor_gen[0][0] and mas_actos[0][1] > mejor_gen[0][1]):
        mejor_gen[0] = mas_actos[0]
        
    poblacion = reproduccion(numDias,numTomasPorDia,num_actos,mas_actos,poblacion)
    poblacion = mutacion(numTomasPorDia,num_alelos_tot,num_max_mutaciones,porcentaje_mutacion,poblacion)

In [53]:
print("pago total: ",mejor_gen[0][0])
print("total intersecciones: ",mejor_gen[0][1])
print("distribucion: ",mejor_gen[0][2])

pago total:  49
total intersecciones:  123
distribucion:  [[6, 21, 29, 13, 9, 19], [10, 14, 4, 25, 0, 20], [23, 2, 17, 27, 15, 18], [26, 7, 11, 28, 8, 1], [5, 24, 16, 22, 12, 3]]


Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta

Primeramente, mejorar la corrección que se hace al momento del cruce, debido a que no puede haber tomas repetidas, estos se están corrigiendo siguiendo un orden del peor alelo al mejor. Sin embargo, en la corrección se asigna una toma aleatoria ahora mismo, no busca aquella que pueda mejora el alelo dentro de las tomas faltantes. Ahora mismo el estado del algoritmo fácilmente se le puede agregar las restricciones de pagos diferentes entre los actores, lo que se acomoda a la vida real. Con respecto al tamaño se puede ampliar la cantidad de días de rodaje siguiendo la simetría con las tomas necesarias, a futuro debería soportar esa asimetría. Con respecto a la cantidad de los actores ahora mismo soporta la variación como se ve dentro del segundo ejemplo. 